In [1]:
from __future__ import print_function
import torch
from torchvision import datasets, transforms
import numpy as np
from WeakLearners import WongNeuralNetCIFAR10
import matplotlib.pyplot as plt
import os

In [2]:
!pip install pytorch_memlab

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from Boosting import Ensemble, runBoosting
from AdversarialAttacks import attack_fgsm, attack_pgd
from pytorch_memlab import LineProfiler

In [4]:
import utils

In [5]:
cuda = torch.device('cuda:0')

In [6]:
%load_ext autoreload
%autoreload 2

## CIFAR10 Boosting (Adversarial)

In [7]:
# maxSamples_vals = [100000, 200000, 500000]
maxSamples_vals = [750000]
# maxSamples_vals = [30001]
# maxSamples_vals = [500000, 1000000]
# done for 50K, 100K
# maxSamples_vals = [1000000, 2000000]
# maxSamples_vals = [123456]
batch_size=100
# maxSamples_vals = [batch_size + 1]

In [8]:
num_wl = 15 # maybe around 100? #later: maybe change this to an array?
num_epochs = 1

In [9]:
ensembles = []
# epsilons = [0.0, 0.01, 0.02, 0.03, 0.05, 0.1]
epsilons = [0.01, 0.02, 0.03]
train_eps_nn = 0
# epsilons = []
# epsilons = [0.0, 0.01]

In [12]:
# from Boosting import SchapireWongMulticlassBoosting
from AdversarialAttacks import attack_fgsm, attack_pgd
import gc
# Ensemble.record_accuracies, Ensemble.calc_accuracies, attack_fgsm, Ensemble.schapireContinuousPredict
for i in range(len(maxSamples_vals)):
    print("len(maxSamples_vals):", len(maxSamples_vals))
    maxSamples = maxSamples_vals[i]
    print("maxSamples:", maxSamples)
    ensemble = runBoosting(num_wl, maxSamples, dataset=datasets.CIFAR10, weakLearnerType = WongNeuralNetCIFAR10, val_attacks = [], 
                               attack_eps_nn=epsilons, attack_eps_ensemble=epsilons, train_eps_nn=train_eps_nn, adv_train_prefix=num_wl, batch_size=batch_size, val_flag=False)
    print("number of wl in ensemble:", len(ensemble.weakLearners))
    ensembles.append(ensemble)
    gc.collect()
#     for obj in gc.get_objects():
#         try:
#             if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
#                 print(type(obj), obj.size())
#         except:
#             pass
    
#     path_head = f'results/plots/cifar10/train_eps_{train_eps_nn}/'
#     try:
#         os.mkdir(path_head)
#     except OSError:
#         pass
#     acc_file = path_head + f'acc_maxSamples_{maxSamples}.png'
#     adv_acc_file = path_head + f'adv_acc_maxSamples_{maxSamples}.png'
#     loss_file = path_head + f'loss_maxSamples_{maxSamples}.png'
#     wl_train_acc_file = path_head + f'wl_train_acc_maxSamples_{maxSamples}.png'
#     ensemble.plot_accuracies(acc_file)
#     ensemble.plot_loss(loss_file)
#     ensemble.plot_adversarial_accuracies(adv_acc_file)
#     ensemble.plot_wl_acc(wl_train_acc_file)


len(maxSamples_vals): 1
maxSamples: 750000
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
attack eps ens [0.01, 0.02, 0.03]
path_head: ./models/750000Eps0/
----------------------------------------------------------------------------------------------------
Training weak learner 0
Epoch 0


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/cifar.py:118: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  img, target = self.data[index], self.targets[index]


Progress: 10000,  val accuracy: 0.3200
Progress: 20000,  val accuracy: 0.4200
Progress: 30000,  val accuracy: 0.5000
Progress: 40000,  val accuracy: 0.4800
Progress: 50000,  val accuracy: 0.5300


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:893: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Epoch 1
Progress: 60000,  val accuracy: 0.5800
Progress: 70000,  val accuracy: 0.5800
Progress: 80000,  val accuracy: 0.6800
Progress: 90000,  val accuracy: 0.6900
Progress: 100000,  val accuracy: 0.7000
Epoch 2
Progress: 110000,  val accuracy: 0.7300
Progress: 120000,  val accuracy: 0.7300
Progress: 130000,  val accuracy: 0.7500
Progress: 140000,  val accuracy: 0.7500
Progress: 150000,  val accuracy: 0.7300
Epoch 3
Progress: 160000,  val accuracy: 0.7600
Progress: 170000,  val accuracy: 0.7300
Progress: 180000,  val accuracy: 0.7700
Progress: 190000,  val accuracy: 0.7400
Progress: 200000,  val accuracy: 0.7500
Epoch 4
Progress: 210000,  val accuracy: 0.7500
Progress: 220000,  val accuracy: 0.8300
Progress: 230000,  val accuracy: 0.7900
Progress: 240000,  val accuracy: 0.8000
Progress: 250000,  val accuracy: 0.8200
Epoch 5
Progress: 260000,  val accuracy: 0.8000
Progress: 270000,  val accuracy: 0.8200
Progress: 280000,  val accuracy: 0.8600
Progress: 290000,  val accuracy: 0.8500
Prog

In [ ]:
from Testing import testEnsemble
path = f'./models/{maxSamples_vals[0]}Eps{train_eps_nn}/'
attack=attack_pgd
attackStr="attack_pgd"
ensemble = testEnsemble(path, [attack], num_wl, numsamples_train=500, numsamples_val=500, attack_eps_ensemble=epsilons)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Weak Learner  0 .  Time Elapsed (s):  0


In [ ]:
maxSamples = maxSamples_vals[0]
resultsPath = f'results/plots/cifar10/train_eps_{train_eps_nn}/{attackStr}/'
acc_file = resultsPath + f'acc_maxSamples_{maxSamples}.png'
adv_acc_file = resultsPath + f'adv_acc_maxSamples_{maxSamples}.png'
loss_file = resultsPath + f'loss_maxSamples_{maxSamples}.png'
# wl_train_acc_file = resultsPath + f'wl_train_acc_maxSamples_{maxSamples}.png'
ensemble.plot_accuracies(acc_file)
ensemble.plot_loss(loss_file)
ensemble.plot_adversarial_accuracies(adv_acc_file)
# ensemble.plot_wl_acc(wl_train_acc_file)


In [ ]:
# for maxSamples in maxSamples_vals:
#         ensemble = runBoosting(num_wl, maxSamples, dataset=datasets.CIFAR10, weakLearnerType = WongNeuralNetCIFAR10, val_attacks = [attack_fgsm], 
#                                attack_eps_nn=epsilons, attack_eps_ensemble=epsilons, train_eps_nn=8, adv_train=True)
#         ensembles.append(ensemble)

In [ ]:
print(ensemble.accuracies)

In [ ]:
ensembles[0].plot_train_loss()
ensembles[0].plot_val_loss()
ensembles[0].plot_train_accuracies()
ensembles[0].plot_val_accuracies()

In [ ]:
ensembles[0].attack_eps

In [ ]:
ensembles[0].plot_wl_acc()

In [ ]:
ensemble

In [ ]:
print("hi")